<a href="https://colab.research.google.com/github/RobAltena/AdventOfCode2022/blob/main/Day12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import queue
import requests
from matplotlib import pyplot as plt

# Sample data.
data = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi""".split('\n')

# The real puzzle.
data = requests.get('https://raw.githubusercontent.com/RobAltena/AdventOfCode2022/main/advent_day12_input.txt').text.split('\n')[:-1]

In [ ]:
grid = np.vstack(list((np.fromiter(line, dtype='c') for line in data)))
plt.imshow(grid.view(np.uint8), cmap="hot")

In [ ]:
border_map = np.pad(grid.view(np.int8), 1, constant_values=-1)
start = tuple(zip(*np.where(border_map == ord('S'))))[0]
end = tuple(zip(*np.where(border_map == ord('E'))))[0]

border_map[start] = ord('a')
border_map[end] = ord('z')

start, end

In [ ]:
directions = [(-1,0), (1,0), (0,-1), (0,1)]

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(start, goal):
    frontier = queue.PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        # Early stopping gets the wrong answer in this one.
        #if current == goal:
            #print('goal found - ', cost_so_far[next])
            #break
        
        for d in directions:
            next = (current[0]+d[0], current[1]+d[1])
            # Stay on the map and do not climb more than one level.
            if (border_map[next] >= 0) and (border_map[next] - border_map[current] <2) :
              new_cost = cost_so_far[current] + 1
              if next not in cost_so_far or new_cost < cost_so_far[next]:
                  cost_so_far[next] = new_cost
                  priority = new_cost + heuristic(next, goal)
                  frontier.put(next, priority)
                  came_from[next] = current
    
    return came_from, cost_so_far

In [ ]:
came_from, cost_so_far = a_star_search(start, end)
print('part 1:', cost_so_far[end])

In [ ]:
# Pretty plot of the route.
x=end
route = []
while(x):
  route.append(x)
  x = came_from[x]

fig, ax = plt.subplots(figsize=(20,20))

ax.imshow(border_map[1:-1,1:-1], cmap="hot")
ax.plot(np.array(route)[:,1]-1,np.array(route)[:,0]-1, color = "black")

In [ ]:
start_locs = tuple(zip(*np.where(border_map == ord('a'))))
steps_required = []
for start in start_locs:
  came_from, cost_so_far = a_star_search(start, end)
  if end in cost_so_far.keys():
    steps_required.append(cost_so_far[end])

print('Part 2: ', min(steps_required))